In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from flopy.utils.gridintersect import GridIntersect
import flopy

# Groundwater Modeling and Python Programming

In this exercise, we will use MODPATH to simulate advective transport with the Freyberg flow model. For this exercise, we will use the flow model created in `07-stream_capture_voronoi`.

The location of the contamination patch and the nodes that the define bounding cells of the patch are calculated below.

In [ ]:
# patch upper left and lower right
xmin, xmax = 250. * 1, 250. * 3
ymin, ymax = (40 - 14) * 250., (40 - 11) * 250. 
# todo: update this to use gridintersect
csx, csy = [xmin, xmin, xmax, xmax, xmin], [ymin, ymax, ymax, ymin, ymin]
polygon = [list(zip(csx, csy))]
(xmin, ymax), (xmax, ymin)

In [ ]:
# patch edge node coordinates
xmin_node, xmax_node = xmin + 125, xmax - 125
ymin_node, ymax_node = ymin + 125, ymax - 125
(xmin_node, ymax_node), (xmax_node, ymin_node)

--------------------------

### Define the workspace and model names.

In [ ]:
load_ws = os.path.join('data', 'quadtree')
ws = os.path.join("temp", "ex10a")
name = "project"
name_mp = name + '_mp'
exe_name = 'mf6'

## Load the MODFLOW 6 Model

Load a simulation object using `flopy.mf6.MFSimulation().load()`.

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_name=name, exe_name=exe_name,
                                    sim_ws=load_ws)

### Load the groundwater flow model

In [ ]:
gwf = sim.get_model(name)
gwf.modelgrid

### Change the workspace

In [ ]:
sim.set_sim_path(ws)

### Write the model files

In [ ]:
sim.write_simulation()

Run the simulation.

In [ ]:
sim.run_simulation()

## Create and Run the MODPATH model


### Lets plot the model grid and the location of the contamination patch.

In [ ]:
fig, ax = plt.subplots(figsize=(5, 9))
mm = flopy.plot.PlotMapView(gwf, layer=0, ax=ax)

mm.plot_bc('SFR', color="b", plotAll=True)
mm.plot_bc('WEL', plotAll=True)
mm.plot_inactive(alpha=0.75)

mm.plot_grid(lw=0.25, color='grey')

ax.fill(csx, csy, color='#e534eb');

Find the node numbers of the contamination patch using the groundwater model grid object and the `GridIntersect` utility. The `GridIntersect` utility has an `intersect` method that can be used to identified cellids from points, lines, and polygons.

In [ ]:
gx = GridIntersect(gwf.modelgrid)
results = gx.intersect(polygon, 'Polygon')
nodes = results.cellids
nodes

### Create the MODPATH particle data

In [ ]:
# create
sd = flopy.modpath.CellDataType()
p = flopy.modpath.NodeParticleData(subdivisiondata=[sd],
                                   nodes=list(nodes))
# create forward particle group
fpth = name_mp + '.sloc'
pg = flopy.modpath.ParticleGroupNodeTemplate(particlegroupname='contaminant',
                                             particledata=p,
                                             filename=fpth)

### Create the MODPATH 7 files

In [ ]:
# create modpath files
mp = flopy.modpath.Modpath7(modelname=name_mp, flowmodel=gwf,
                            exe_name='mp7', model_ws=ws)
mpbas = flopy.modpath.Modpath7Bas(mp, porosity=0.1)
mpsim = flopy.modpath.Modpath7Sim(mp, simulationtype='pathline', 
                                  trackingdirection='forward',
                                  weaksinkoption='pass_through',
                                  weaksourceoption='pass_through',
                                  referencetime=0.,
                                  stoptimeoption='extend',
                                  particlegroups=pg)

### Write MODPATH 7 files and run the model

In [ ]:
# write modpath datasets
mp.write_input()

# run modpath
mp.run_model()

## Post-Process the MODFLOW and MODPATH Results


### Load MODFLOW and MODPATH results from the heads and pathline files

Load the MODFLOW heads

In [ ]:
hname = gwf.oc.head_filerecord.array[0][0]
hname

In [ ]:
fpth = os.path.join(ws, hname)
hobj = flopy.utils.HeadFile(fpth)
hds = hobj.get_data()

Load the pathline file

In [ ]:
ppth = os.path.join(ws, name_mp + '.mppth')
p = flopy.utils.PathlineFile(ppth)
p0 = p.get_alldata()

In [ ]:
#p0

### Plot the heads and pathlines

In [ ]:
fig, ax = plt.subplots(figsize=(5, 9))
mm = flopy.plot.PlotMapView(model=gwf, layer=0, ax=ax)
mm.plot_array(hds, masked_values=[1e30])

mm.plot_bc('SFR', color='b', plotAll=True)
mm.plot_bc('WEL', plotAll=True)
mm.plot_ibound()
mm.plot_pathline(p0, layer='all', color='blue', lw=0.75)

ax = plt.gca()
ax.fill(csx, csy, color='#e534eb', zorder=100, alpha=.75);

### Export the pathlines to a shapefile

In [ ]:
spth = os.path.join(ws, 'pathline.shp')
p.write_shapefile(p0, mg=gwf.modelgrid, one_per_particle=False, shpname=spth)

### Load the shapefile into geopandas

In [ ]:
rshp = gpd.read_file(spth)

### Show the dataframe

In [ ]:
rshp

### Plot the geopandas dataframe on the modelgrid

In [ ]:
fig, ax = plt.subplots(figsize=(6, 10))
pmv = flopy.plot.PlotMapView(gwf, ax=ax)
pmv.plot_grid(lw=0.5)
pmv.plot_shapes(rshp.geometry.to_list(), edgecolor="grey") #, edgecolor="b")
pmv.plot_ibound();